In [53]:
using Turing
using ReverseDiff
Turing.setadbackend(:reversediff)
using LimberJack
using LinearAlgebra
using CSV
using NPZ
using Plots
using FITSIO
using ForwardDiff
using BenchmarkTools
using DelimitedFiles

In [54]:
files = npzread("../data/DESY1_cls/wlwl.npz");
nz_path = "../data/DESY1_cls/fullmarg_nzs/"

"../data/DESY1_cls/fullmarg_nzs/"

In [55]:
cov_tot = files["cov"]
inv_cov_tot = files["inv_cov"]
data_vector = files["cls"];

In [56]:
zs_k0, nz_k0, cov_k0 = get_nzs(nz_path, 2, 0)
zs_k1, nz_k1, cov_k1 = get_nzs(nz_path, 2, 1)
zs_k2, nz_k2, cov_k2 = get_nzs(nz_path, 2, 2)
zs_k3, nz_k3, cov_k3 = get_nzs(nz_path, 2, 3);

In [57]:
cov_k0 = Diagonal(cov_k0)
cov_k1 = Diagonal(cov_k1)
cov_k2 = Diagonal(cov_k2)
cov_k3 = Diagonal(cov_k3)

100×100 Diagonal{Float64, Vector{Float64}}:
 1.6708e-8   ⋅           ⋅           …   ⋅          ⋅          ⋅ 
  ⋅         5.80425e-5   ⋅               ⋅          ⋅          ⋅ 
  ⋅          ⋅          0.000881997      ⋅          ⋅          ⋅ 
  ⋅          ⋅           ⋅               ⋅          ⋅          ⋅ 
  ⋅          ⋅           ⋅               ⋅          ⋅          ⋅ 
  ⋅          ⋅           ⋅           …   ⋅          ⋅          ⋅ 
  ⋅          ⋅           ⋅               ⋅          ⋅          ⋅ 
  ⋅          ⋅           ⋅               ⋅          ⋅          ⋅ 
  ⋅          ⋅           ⋅               ⋅          ⋅          ⋅ 
  ⋅          ⋅           ⋅               ⋅          ⋅          ⋅ 
  ⋅          ⋅           ⋅           …   ⋅          ⋅          ⋅ 
  ⋅          ⋅           ⋅               ⋅          ⋅          ⋅ 
  ⋅          ⋅           ⋅               ⋅          ⋅          ⋅ 
 ⋮                                   ⋱                        
  ⋅          ⋅           ⋅         

In [58]:
#                Omega_b,           Omega_c          n_s,                 h,
#                DESgc__0_gc_b,     DESgc__1_gc_b,   DESgc__2_gc_b,    DESgc__3_gc_b,    DESgc__4_gc_b,
#                DESgc__0_dz,      DESgc__1_dz,      DESgc__2_dz,      DESgc__3_dz,      DESgc__4_dz,
#                wl_ia_A,          wl_ia_eta, 
#                DESwl__0_wl_m,    DESwl__1_wl_m,    DESwl__2_wl_m,    DESwl__3_wl_m,
#                DESwl__0_dz,      DESwl__1_dz,      DESwl__2_dz,      DESwl__3_dz,
#                sigma_8,          Omega_m
pars = [4.426868e-02,     2.093138e-01,     8.963611e-01,     8.495440e-01,
         1.343888e+00,    1.639047e+00,      1.597174e+00,     1.944583e+00,     2.007245e+00,
        -4.679383e-03,   -2.839996e-03,      1.771571e-03,     1.197051e-03,    -5.199799e-03,
         2.389208e-01,   -6.435288e-01, 
         1.802722e-03,   -5.508994e-03,     1.952514e-02,    -1.117726e-03,
        -1.744083e-02,    6.777779e-03,    -1.097939e-03,    -4.912315e-03,
         8.536883e-01,    2.535825e-01];

In [37]:
nuisances = Dict("b0" => pars[5],
                 "b1" => pars[6],
                 "b2" => pars[7],
                 "b3" => pars[8],
                 "b4" => pars[9],
                 "dz_g0" => pars[10],
                 "dz_g1" => pars[11],
                 "dz_g2" => pars[12],
                 "dz_g3" => pars[13],
                 "dz_g4" => pars[14],
                 "dz_k0" => pars[21],
                 "dz_k1" => pars[22],
                 "dz_k2" => pars[23],
                 "dz_k3" => pars[24],
                 "m0" => pars[17],
                 "m1" => pars[18],
                 "m2" => pars[19],
                 "m3" => pars[20],
                 "A_IA" => pars[15],
                 "alpha_IA" => pars[16]);

In [38]:
@model function model(data_vector; cov_tot=cov_tot, nz_path=nz_path)
    #KiDS priors
    Ωm ~ Uniform(0.1, 0.6)
    Ωb = pars[1]
    h = pars[4]
    ns = pars[3]
    s8 = pars[25]
    
    Nz_k0 ~ MvNormal(nz_k0, cov_k0)
    Nz_k1 ~ MvNormal(nz_k1, cov_k1)
    Nz_k2 ~ MvNormal(nz_k2, cov_k2)
    Nz_k3 ~ MvNormal(nz_k3, cov_k3)

    nuisances = Dict("nz_k0" => Nz_k0,
                     "nz_k1" => Nz_k1,
                     "nz_k2" => Nz_k2,
                     "nz_k3" => Nz_k3,
                     "m0" => pars[17],
                     "m1" => pars[18],
                     "m2" => pars[19],
                     "m3" => pars[20],
                     "A_IA" => pars[15],
                     "alpha_IA" => pars[16])
    
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    
    theory = Theory(cosmology, files;
                    nz_path=nz_path,
                    Nuisances=nuisances)
    
    data_vector ~ MvNormal(theory, cov_tot)
end;

In [39]:
iterations = 300
nadapts = 100;

In [40]:
chain3 = sample(model(data_vector), NUTS(5, 0.1; init_ϵ=0.03), iterations,
                progress=true; save_state=true);

LoadError: PosDefException: matrix is not positive definite; Cholesky factorization failed.

In [32]:
chain1 = sample(model(data_vector), MH(), iterations,
            nadapt=nadapts, progress=true; save_state=true);

LoadError: MethodError: no method matching PDMats.PDMat(::Hermitian{Float64, Matrix{Float64}})
[0mClosest candidates are:
[0m  PDMats.PDMat(::AbstractMatrix, [91m::Cholesky{T, S}[39m) where {T, S} at ~/.julia/packages/PDMats/G0Prn/src/pdmat.jl:12
[0m  PDMats.PDMat([91m::Cholesky[39m) at ~/.julia/packages/PDMats/G0Prn/src/pdmat.jl:21
[0m  PDMats.PDMat([91m::Symmetric[39m) at ~/.julia/packages/PDMats/G0Prn/src/pdmat.jl:20
[0m  ...